## Simulating encounters

If this is your first time using a notebook, please ensure you have [Python installed](https://www.python.org/downloads/) and you have installed the additional dependencies with the following command: `pip install -r ./automation/requirements.txt`

First, we'll navigate to the repo's root:


In [1]:
import os

if os.path.basename(os.getcwd()) != "TheGame":
    os.chdir("..")

Next, import what you need:


In [2]:
import copy
from automation.templates.bestiary import Bestiary
from automation.simulator.deck import Deck
from automation.simulator.encounter import Encounter

Use the Bestiary to provide dictionaries for each of the combatants. We can use emojis to represent them in the combat output. The Name value will be shown in the log.

Note that files with SAMPLE in the name are designed to workshop ideas and test edge cases, not be included in the game.


In [3]:
b = Bestiary(input_files=["06_Bestiary_SAMPLE.yaml","07_PCs.yaml"]).raw_data
c1 = b["Clubs1"]
c1.update(dict(Name="💀", id="A"))
c2 = copy.copy(b["Clubs1"])
c2.update(dict(Name="👽", id="B"))
s = b["Spider Queen"]
s.update(dict(Name="🎇", id="C"))

Initialize the encounter. We can see who is involved and resource information by looking at the encounter's `PCs`, `enemies`, or `turn_order` properties.

Note that this may differ from a real encounter because each participant has their own hand and deck. In a true encounter, a GM might manage many characters with the same deck.


In [4]:
e = Encounter(PCs=[c1, c2], Enemies=[s])
e.turn_order

[01:44][WARNING]: 💀 used 1 excess XP
[01:44][WARNING]: 👽 used 1 excess XP


[💀
 TC       :♠️ A | pc.HP : 6/6
 Hand     :  03 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 11/11
 Discards :  00 | RestC : 6/6,
 👽
 TC       :♥️ 7 | pc.HP : 6/6
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 11/11
 Discards :  01 | RestC : 6/6,
 🎇
 TC       :♠️ 3 | pc.HP : 8/8
 Hand     :  02 | pc.PP : 0/0
 Deck     :  51 | pc.AP : 3/3
 Discards :  01 | RestC : 8/8]

Next, we can simulate a couple rounds of combat. Here, each participant will choose a Power available to them (if sufficient PP) and apply it to an enemy at random. This does not yet cover buffing powers (e.g., Shield, Lend Aid) or mind control status effects (i.e., Charmed, Enthralled).


In [5]:
e.sim_round(3)

[01:44][INFO]: 💀 used 1/2 PP with Shield, Self


TypeError: can only concatenate str (not "int") to str

In [6]:
%debug

> /Users/cb/Documents/temp/creative/TheGame/automation/simulator/player.py(71)_apply_upper_lower()
     69             extra -= 1
     70         kwarg_dict["upper_lower_int"] = (
---> 71             kwarg_dict.get("upper_lower_int", 0)
     72             + self._statuses.get(all_type, 0)
     73             + self._statuses.get(next_type, 0)

> /Users/cb/Documents/temp/creative/TheGame/automation/simulator/player.py(114)check_by_skill()
    112             kwargs["return_val"] = True
    113         skill, mod = self._find_highest_stat(skill)
--> 114         new_kwargs = self._apply_upper_lower("check", kwargs, skill=skill)
    115         result = self.check(TC, DR, mod=mod, **new_kwargs)
    116         if result == 0:

*** NameError: name 'kwags' is not defined
> /Users/cb/Documents/temp/creative/TheGame/automation/simulator/player.py(71)_apply_upper_lower()
     69             extra -= 1
     70         kwarg_dict["upper_lower_int"] = (
---> 71             kwarg_dict.get("upper_l

We can even simulate epic events.


In [ ]:
e.sim_epic_event(successes_needed=3)

Let's see how everyone is doing.


In [ ]:
e.turn_order

If you're interested in draining a specific character's resources, we can index them directly and subtract values.


In [ ]:
spider_queen = e.enemies[0]
spider_queen.HP = 1
spider_queen

The only thing that can't be set directly is the deck. This has to be managed by either drawing, using a fate card, or shuffling.


In [ ]:
spider_queen.draw()

In [ ]:
spider_queen.exchange_fate()

In [ ]:
spider_queen.shuffle(2)
spider_queen

We can give some or all participants a rest.


In [ ]:
e.sim_quick_rest(participants=e.PCs)  # If no participants specified, all

In [ ]:
e.turn_order

Or a full rest.


In [ ]:
e.sim_full_rest()

In [ ]:
e.turn_order

## Logging

Much of the information we're seeing above in the `[TIME][INFO]` format is as a result of the [logger](https://www.loggly.com/ultimate-guide/python-logging-basics/) that keeps track of this information. We can adjust what we see by adjusting the log level.


In [ ]:
from automation.utils import logger

logger.setLevel("DEBUG")  # Most information
# logger.setLevel('INFO') # Standard information
# logger.setLevel('WARNING') # Only problems
# logger.setLevel('CRITICAL') # No information
spider_queen.save(DR=3, attrib=["AGL", "STR"], upper_lower="lower", draw_n=2)
e.sim_round()

For a more detailed record of checks, saves, and rests, turn on CSV logging. This will save logs of who performed which check/save or rest and some associated values.


In [ ]:
e.set_csv_logging(True)
e.sim_round()
e.sim_quick_rest()

The following uses system commands rather than Python to look at the output. To use this data more meaningfully, try loading the data with [pandas](https://pythonbasics.org/read-csv-with-pandas/).


In [ ]:
!head automation/_output/log_draws.csv

In [ ]:
!head automation/_output/log_rests.csv